In [30]:
# Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
# 체인을 수동으로 구현해야 합니다.
# 체인에 ConversationBufferMemory를 부여합니다.
# 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
# 체인에 다음 질문을 합니다:
    # Aaronson 은 유죄인가요?
    # 그가 테이블에 어떤 메시지를 썼나요?
    # Julia 는 누구인가요?


from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
   temperature=0.1,
#    streaming=True,
#    callbacks=[StreamingStdOutCallbackHandler()],
)

loader = UnstructuredFileLoader("./files/document.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
    # length_function=len,
)

docs = loader.load_and_split(text_splitter=splitter)
# print(docs)

cache_dir = LocalFileStore("./cache/")
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir,)

vector_store = FAISS.from_documents(docs, cached_embeddings)

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="history",
    return_messages=True,
)

retriever = vector_store.as_retriever()

# 메모리 로드 함수 정의
def load_memory(input):
    print("loadmemory()>> input:",input)
    return memory.load_memory_variables({})['history']

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assistant. Answer questions using only the following context.If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# 체인 설정
chain = {"context": retriever,"question": RunnablePassthrough(),"history": load_memory,} | prompt | llm

def invoke_chain(question):
    # result = chain.invoke({"question": question,}) # >> 이렇게하면 expected string or buffer가 나옴
    result = chain.invoke(question)
    memory.save_context(
        {"input":question},
        {"output":result.content}
    )
    print(result)


In [31]:
# result = chain.invoke("Is Aaronson guilty?")
# print(result)
# result = chain.invoke("What message did he write in the table?")
# print(result)
# result = chain.invoke("Who is Julia?")
# print(result)

invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

loadmemory()>> input: Is Aaronson guilty?
content='Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'
loadmemory()>> input: What message did he write in the table?
content='He wrote "2+2=5" in the dust on the table.'
loadmemory()>> input: Who is Julia?
content='Julia is a character in the text who is associated with the protagonist and plays a significant role in the story.'


In [35]:
load_memory({}) #변수는 아무거나 넣어도 됨

loadmemory()>> input: [HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'), HumanMessage(content='What message did he write in the table?'), AIMessage(content='He wrote "2+2=5" in the dust on the table.'), HumanMessage(content='Who is Julia?'), AIMessage(content='Julia is a character in the text who is associated with the protagonist and plays a significant role in the story.')]


[HumanMessage(content='Is Aaronson guilty?'),
 AIMessage(content='Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'),
 HumanMessage(content='What message did he write in the table?'),
 AIMessage(content='He wrote "2+2=5" in the dust on the table.'),
 HumanMessage(content='Who is Julia?'),
 AIMessage(content='Julia is a character in the text who is associated with the protagonist and plays a significant role in the story.')]